<div style="background: #000;
            color: #FFF;
            margin: 0px;
            padding: 10px 0px 20px 0px;
            text-align: center; 
                ">
    <h1>Week 18 Lab</h1>
</div>

For today's lab, we'll be using the Marvel comic book characters data set available from [538](https://github.com/fivethirtyeight/data/tree/master/comic-characters)

The data comes from [Marvel Wikia](http://marvel.wikia.com/Main_Page) and [DC Wikia](http://dc.wikia.com/wiki/Main_Page). Characters were scraped on August 24. Appearance counts were scraped on September 2. The month and year of the first issue each character appeared in was pulled on October 6.

The data is split into two files, for DC and Marvel, respectively: `dc-wikia-data.csv` and `marvel-wikia-data.csv`. Each file has the following variables:

Variable | Definition
---|---------
`page_id` | The unique identifier for that characters page within the wikia
`name` | The name of the character
`urlslug` | The unique url within the wikia that takes you to the character
`ID` | The identity status of the character (Secret Identity, Public identity, [on marvel only: No Dual Identity])
`ALIGN` | If the character is Good, Bad or Neutral
`EYE` | Eye color of the character
`HAIR` | Hair color of the character
`SEX` | Sex of the character (e.g. Male, Female, etc.)
`GSM` | If the character is a gender or sexual minority (e.g. Homosexual characters, bisexual characters)
`ALIVE` | If the character is alive or deceased
`APPEARANCES` | The number of appareances of the character in comic books (as of Sep. 2, 2014. Number will become increasingly out of date as time goes on.)
`FIRST APPEARANCE` | The month and year of the character's first appearance in a comic book, if available
`YEAR` | The year of the character's first appearance in a comic book, if available


Importing the libraries we'll be using

In [1]:
import numpy as np 
import pandas as pd 
import datetime

Let's open the `marvel-wikia-data.csv` file and generate a dataframe. Then we'll check the top of the `df` to see what we have.

In [127]:
df = pd.read_csv("../datasets/marvel-wikia-data.csv")
df

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach (Earth-616),\/Ru%27ach_(Earth-616),No Dual Identity,Bad Characters,Green Eyes,No Hair,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16372,665474,Thane (Thanos' son) (Earth-616),\/Thane_(Thanos%27_son)_(Earth-616),No Dual Identity,Good Characters,Blue Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16373,695217,Tinkerer (Skrull) (Earth-616),\/Tinkerer_(Skrull)_(Earth-616),Secret Identity,Bad Characters,Black Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16374,708811,TK421 (Spiderling) (Earth-616),\/TK421_(Spiderling)_(Earth-616),Secret Identity,Neutral Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN


As you can see we have a couple of issues:
* The column `names` has, in some cases, both the hero name and their secret identity. In other cases, it has the universe the hero is from (usually Earth-616). We'd like for `name` to be the superhero. If a secret identity name is available, we'd like for that to be in it's own column. We'd also like a column `origin` for the superhero's place of origin (usually Earth-616).
* The `urlslug` column is incorrect. We'd like for that to be a complete link. For example, spiderman's wikia link is https://marvel.fandom.com/wiki/Peter_Parker_(Earth-616) but if you use the `Spider-Man_(Peter_Parker)` from the `urlslug` (https://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker), the page redirects to the correct page. You can choose to fix it various different ways but it should be a working link. Use "https://marvel.fandom.com/wiki/" as the beginning of your url.
* For the column `align`, we'd like for the "Characters" part to be removed. We'd like for that column to be categorical (Good, Neutral, Evil or blank).
* Same for the columns `EYE`, `HAIR`, `SEX` and `ID`. We want to drop the extraneous data and have simple categories (e.g. for `ID` we want categories ["Known","Public", "Secret", "None"] and for `SEX` we just want ["Male", "Female","Genderfluid","Agender"]. There's a way shown below that shows the various possible categories.
* Some data is in the wrong format. For example, `Year` and `APPEARANCES` are floats but they really should be ints (as you cannot have a non-int number of appearances). 
* Some data doesn't match up (like the `FIRST APPEARANCE` and `Year` columns might have not be the same year)
* Some columns are unnecessary. Some are missing data. Some rows are duplicates. Fix those as necessary.
* Some column names are written as uppercase, some are lowercase and some are mixed case. Normalize them so all the column names are in the same format.

Fix as many of these issues as you're able to. At the minimum, complete 4 of the above bulletpoints.

In [128]:
def see_possible_categories(col):
    return df.groupby(col).sum()
    
see_possible_categories('name')

,page_id,APPEARANCES,Year
name,,,
'Spinner (Earth-616),322600,0.0,2007.0
107 (Earth-616),116257,1.0,1994.0
11-Ball (Earth-616),543479,1.0,1991.0
115 (Legion Personality) (Earth-616),624448,1.0,2010.0
14 (Earth-616),116253,1.0,1994.0
...,...,...,...
Zzxz (Earth-616),250460,6.0,2009.0
Zzzax (Earth-616),1553,32.0,1973.0
"\""Spider-Girl\"" (Mutant\/Spider Clone) (Earth-616)",689359,0.0,2013.0


In [129]:
df.shape

(16376, 13)

## We'd like for name to be the superhero. If a secret identity name is available, we'd like for that to be in it's own column. We'd also like a column origin for the superhero's place of origin (usually Earth-616).

In [130]:
df["Secret_Identity"] = df['name'].str.split('(',3).str[1]


In [131]:
df["Secret_Identity"]=df["Secret_Identity"].str.split(")").str[0]
df

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,Secret_Identity
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0,Peter Parker
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0,Steven Rogers
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0,"James \""Logan\"" Howlett"
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0,"Anthony \""Tony\"" Stark"
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0,Thor Odinson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach (Earth-616),\/Ru%27ach_(Earth-616),No Dual Identity,Bad Characters,Green Eyes,No Hair,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Earth-616
16372,665474,Thane (Thanos' son) (Earth-616),\/Thane_(Thanos%27_son)_(Earth-616),No Dual Identity,Good Characters,Blue Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Thanos' son
16373,695217,Tinkerer (Skrull) (Earth-616),\/Tinkerer_(Skrull)_(Earth-616),Secret Identity,Bad Characters,Black Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Skrull
16374,708811,TK421 (Spiderling) (Earth-616),\/TK421_(Spiderling)_(Earth-616),Secret Identity,Neutral Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Spiderling


In [132]:
df["Origin"] = df['name'].str.split('(',3).str[2]
df

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,Secret_Identity,Origin
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0,Peter Parker,NaN
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0,Steven Rogers,NaN
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0,"James \""Logan\"" Howlett",NaN
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0,"Anthony \""Tony\"" Stark",NaN
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0,Thor Odinson,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach (Earth-616),\/Ru%27ach_(Earth-616),No Dual Identity,Bad Characters,Green Eyes,No Hair,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Earth-616,NaN
16372,665474,Thane (Thanos' son) (Earth-616),\/Thane_(Thanos%27_son)_(Earth-616),No Dual Identity,Good Characters,Blue Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Thanos' son,Earth-616)
16373,695217,Tinkerer (Skrull) (Earth-616),\/Tinkerer_(Skrull)_(Earth-616),Secret Identity,Bad Characters,Black Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Skrull,Earth-616)
16374,708811,TK421 (Spiderling) (Earth-616),\/TK421_(Spiderling)_(Earth-616),Secret Identity,Neutral Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Spiderling,Earth-616)


In [133]:
df["name"] = df['name'].str.split('(',3).str[0]
df

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,Secret_Identity,Origin
0,1678,Spider-Man,\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0,Peter Parker,NaN
1,7139,Captain America,\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0,Steven Rogers,NaN
2,64786,Wolverine,\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0,"James \""Logan\"" Howlett",NaN
3,1868,Iron Man,\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0,"Anthony \""Tony\"" Stark",NaN
4,2460,Thor,\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0,Thor Odinson,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach,\/Ru%27ach_(Earth-616),No Dual Identity,Bad Characters,Green Eyes,No Hair,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Earth-616,NaN
16372,665474,Thane,\/Thane_(Thanos%27_son)_(Earth-616),No Dual Identity,Good Characters,Blue Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Thanos' son,Earth-616)
16373,695217,Tinkerer,\/Tinkerer_(Skrull)_(Earth-616),Secret Identity,Bad Characters,Black Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Skrull,Earth-616)
16374,708811,TK421,\/TK421_(Spiderling)_(Earth-616),Secret Identity,Neutral Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Spiderling,Earth-616)


## The urlslug column is incorrect. We'd like for that to be a complete link. For example, spiderman's wikia link is https://marvel.fandom.com/wiki/Peter_Parker_(Earth-616) but if you use the Spider-Man_(Peter_Parker) from the urlslug (https://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker), the page redirects to the correct page. You can choose to fix it various different ways but it should be a working link. Use "https://marvel.fandom.com/wiki/" as the beginning of your url.

In [134]:
#df["urlslug"]

In [135]:
df["urlslug"] = "https://marvel.fandom.com/wiki/" + df["urlslug"].str.split("/").str[1]


In [136]:
df["urlslug"]

0        https://marvel.fandom.com/wiki/Spider-Man_(Pet...
1        https://marvel.fandom.com/wiki/Captain_America...
2        https://marvel.fandom.com/wiki/Wolverine_(Jame...
3        https://marvel.fandom.com/wiki/Iron_Man_(Antho...
4        https://marvel.fandom.com/wiki/Thor_(Thor_Odin...
                               ...                        
16371    https://marvel.fandom.com/wiki/Ru%27ach_(Earth...
16372    https://marvel.fandom.com/wiki/Thane_(Thanos%2...
16373    https://marvel.fandom.com/wiki/Tinkerer_(Skrul...
16374    https://marvel.fandom.com/wiki/TK421_(Spiderli...
16375    https://marvel.fandom.com/wiki/Yologarch_(Eart...
Name: urlslug, Length: 16376, dtype: object

In [137]:
df

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,Secret_Identity,Origin
0,1678,Spider-Man,https://marvel.fandom.com/wiki/Spider-Man_(Pet...,Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0,Peter Parker,NaN
1,7139,Captain America,https://marvel.fandom.com/wiki/Captain_America...,Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0,Steven Rogers,NaN
2,64786,Wolverine,https://marvel.fandom.com/wiki/Wolverine_(Jame...,Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0,"James \""Logan\"" Howlett",NaN
3,1868,Iron Man,https://marvel.fandom.com/wiki/Iron_Man_(Antho...,Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0,"Anthony \""Tony\"" Stark",NaN
4,2460,Thor,https://marvel.fandom.com/wiki/Thor_(Thor_Odin...,No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0,Thor Odinson,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach,https://marvel.fandom.com/wiki/Ru%27ach_(Earth...,No Dual Identity,Bad Characters,Green Eyes,No Hair,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Earth-616,NaN
16372,665474,Thane,https://marvel.fandom.com/wiki/Thane_(Thanos%2...,No Dual Identity,Good Characters,Blue Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Thanos' son,Earth-616)
16373,695217,Tinkerer,https://marvel.fandom.com/wiki/Tinkerer_(Skrul...,Secret Identity,Bad Characters,Black Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Skrull,Earth-616)
16374,708811,TK421,https://marvel.fandom.com/wiki/TK421_(Spiderli...,Secret Identity,Neutral Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Spiderling,Earth-616)


## For the column align, we'd like for the "Characters" part to be removed. We'd like for that column to be categorical (Good, Neutral, Evil or blank).

In [138]:
df['ALIGN'] = df['ALIGN'].str.split('C').str[0]

In [139]:
df["ALIGN"] = df["ALIGN"].replace(["Bad "], "Evil")

In [140]:
df["ALIGN"].isna().sort_values(ascending=False) #which Align values are empty

11838     True
2244      True
2227      True
13469     True
2228      True
         ...  
10316    False
10315    False
10314    False
10313    False
0        False
Name: ALIGN, Length: 16376, dtype: bool

In [141]:
df["ALIGN"].isna().sum()  # how many Align values are empty

2812

In [142]:
def see_possible_categories(col):
    return df.groupby(col).sum()
    
see_possible_categories("ALIGN")

,page_id,APPEARANCES,Year
ALIGN,,,
Evil,2030768533,52910.0,12780912.0
Good,1310680907,150603.0,8670851.0
Neutral,771906748,41091.0,4122737.0


In [143]:
df

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,Secret_Identity,Origin
0,1678,Spider-Man,https://marvel.fandom.com/wiki/Spider-Man_(Pet...,Secret Identity,Good,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0,Peter Parker,NaN
1,7139,Captain America,https://marvel.fandom.com/wiki/Captain_America...,Public Identity,Good,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0,Steven Rogers,NaN
2,64786,Wolverine,https://marvel.fandom.com/wiki/Wolverine_(Jame...,Public Identity,Neutral,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0,"James \""Logan\"" Howlett",NaN
3,1868,Iron Man,https://marvel.fandom.com/wiki/Iron_Man_(Antho...,Public Identity,Good,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0,"Anthony \""Tony\"" Stark",NaN
4,2460,Thor,https://marvel.fandom.com/wiki/Thor_(Thor_Odin...,No Dual Identity,Good,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0,Thor Odinson,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach,https://marvel.fandom.com/wiki/Ru%27ach_(Earth...,No Dual Identity,Evil,Green Eyes,No Hair,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Earth-616,NaN
16372,665474,Thane,https://marvel.fandom.com/wiki/Thane_(Thanos%2...,No Dual Identity,Good,Blue Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Thanos' son,Earth-616)
16373,695217,Tinkerer,https://marvel.fandom.com/wiki/Tinkerer_(Skrul...,Secret Identity,Evil,Black Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Skrull,Earth-616)
16374,708811,TK421,https://marvel.fandom.com/wiki/TK421_(Spiderli...,Secret Identity,Neutral,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Spiderling,Earth-616)


## Same for the columns EYE, HAIR, SEX and ID. We want to drop the extraneous data and have simple categories (e.g. for ID we want categories ["Known","Public", "Secret", "None"] and for SEX we just want ["Male", "Female","Genderfluid","Agender"]. There's a way shown below that shows the various possible categories.

In [144]:
df['EYE'] = df['EYE'].str.split('E').str[0]
df

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,Secret_Identity,Origin
0,1678,Spider-Man,https://marvel.fandom.com/wiki/Spider-Man_(Pet...,Secret Identity,Good,Hazel,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0,Peter Parker,NaN
1,7139,Captain America,https://marvel.fandom.com/wiki/Captain_America...,Public Identity,Good,Blue,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0,Steven Rogers,NaN
2,64786,Wolverine,https://marvel.fandom.com/wiki/Wolverine_(Jame...,Public Identity,Neutral,Blue,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0,"James \""Logan\"" Howlett",NaN
3,1868,Iron Man,https://marvel.fandom.com/wiki/Iron_Man_(Antho...,Public Identity,Good,Blue,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0,"Anthony \""Tony\"" Stark",NaN
4,2460,Thor,https://marvel.fandom.com/wiki/Thor_(Thor_Odin...,No Dual Identity,Good,Blue,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0,Thor Odinson,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach,https://marvel.fandom.com/wiki/Ru%27ach_(Earth...,No Dual Identity,Evil,Green,No Hair,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Earth-616,NaN
16372,665474,Thane,https://marvel.fandom.com/wiki/Thane_(Thanos%2...,No Dual Identity,Good,Blue,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Thanos' son,Earth-616)
16373,695217,Tinkerer,https://marvel.fandom.com/wiki/Tinkerer_(Skrul...,Secret Identity,Evil,Black,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Skrull,Earth-616)
16374,708811,TK421,https://marvel.fandom.com/wiki/TK421_(Spiderli...,Secret Identity,Neutral,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Spiderling,Earth-616)


In [145]:
df['HAIR'] = df['HAIR'].str.split('H').str[0]
df

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,Secret_Identity,Origin
0,1678,Spider-Man,https://marvel.fandom.com/wiki/Spider-Man_(Pet...,Secret Identity,Good,Hazel,Brown,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0,Peter Parker,NaN
1,7139,Captain America,https://marvel.fandom.com/wiki/Captain_America...,Public Identity,Good,Blue,White,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0,Steven Rogers,NaN
2,64786,Wolverine,https://marvel.fandom.com/wiki/Wolverine_(Jame...,Public Identity,Neutral,Blue,Black,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0,"James \""Logan\"" Howlett",NaN
3,1868,Iron Man,https://marvel.fandom.com/wiki/Iron_Man_(Antho...,Public Identity,Good,Blue,Black,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0,"Anthony \""Tony\"" Stark",NaN
4,2460,Thor,https://marvel.fandom.com/wiki/Thor_(Thor_Odin...,No Dual Identity,Good,Blue,Blond,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0,Thor Odinson,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach,https://marvel.fandom.com/wiki/Ru%27ach_(Earth...,No Dual Identity,Evil,Green,No,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Earth-616,NaN
16372,665474,Thane,https://marvel.fandom.com/wiki/Thane_(Thanos%2...,No Dual Identity,Good,Blue,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Thanos' son,Earth-616)
16373,695217,Tinkerer,https://marvel.fandom.com/wiki/Tinkerer_(Skrul...,Secret Identity,Evil,Black,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Skrull,Earth-616)
16374,708811,TK421,https://marvel.fandom.com/wiki/TK421_(Spiderli...,Secret Identity,Neutral,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Spiderling,Earth-616)


In [146]:
df['SEX'] = df['SEX'].str.split('C').str[0]

In [147]:
df

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,Secret_Identity,Origin
0,1678,Spider-Man,https://marvel.fandom.com/wiki/Spider-Man_(Pet...,Secret Identity,Good,Hazel,Brown,Male,NaN,Living Characters,4043.0,Aug-62,1962.0,Peter Parker,NaN
1,7139,Captain America,https://marvel.fandom.com/wiki/Captain_America...,Public Identity,Good,Blue,White,Male,NaN,Living Characters,3360.0,Mar-41,1941.0,Steven Rogers,NaN
2,64786,Wolverine,https://marvel.fandom.com/wiki/Wolverine_(Jame...,Public Identity,Neutral,Blue,Black,Male,NaN,Living Characters,3061.0,Oct-74,1974.0,"James \""Logan\"" Howlett",NaN
3,1868,Iron Man,https://marvel.fandom.com/wiki/Iron_Man_(Antho...,Public Identity,Good,Blue,Black,Male,NaN,Living Characters,2961.0,Mar-63,1963.0,"Anthony \""Tony\"" Stark",NaN
4,2460,Thor,https://marvel.fandom.com/wiki/Thor_(Thor_Odin...,No Dual Identity,Good,Blue,Blond,Male,NaN,Living Characters,2258.0,Nov-50,1950.0,Thor Odinson,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach,https://marvel.fandom.com/wiki/Ru%27ach_(Earth...,No Dual Identity,Evil,Green,No,Male,NaN,Living Characters,NaN,NaN,NaN,Earth-616,NaN
16372,665474,Thane,https://marvel.fandom.com/wiki/Thane_(Thanos%2...,No Dual Identity,Good,Blue,Bald,Male,NaN,Living Characters,NaN,NaN,NaN,Thanos' son,Earth-616)
16373,695217,Tinkerer,https://marvel.fandom.com/wiki/Tinkerer_(Skrul...,Secret Identity,Evil,Black,Bald,Male,NaN,Living Characters,NaN,NaN,NaN,Skrull,Earth-616)
16374,708811,TK421,https://marvel.fandom.com/wiki/TK421_(Spiderli...,Secret Identity,Neutral,NaN,NaN,Male,NaN,Living Characters,NaN,NaN,NaN,Spiderling,Earth-616)


In [148]:
df['ALIVE'] = df['ALIVE'].str.split('C').str[0]
df

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,Secret_Identity,Origin
0,1678,Spider-Man,https://marvel.fandom.com/wiki/Spider-Man_(Pet...,Secret Identity,Good,Hazel,Brown,Male,NaN,Living,4043.0,Aug-62,1962.0,Peter Parker,NaN
1,7139,Captain America,https://marvel.fandom.com/wiki/Captain_America...,Public Identity,Good,Blue,White,Male,NaN,Living,3360.0,Mar-41,1941.0,Steven Rogers,NaN
2,64786,Wolverine,https://marvel.fandom.com/wiki/Wolverine_(Jame...,Public Identity,Neutral,Blue,Black,Male,NaN,Living,3061.0,Oct-74,1974.0,"James \""Logan\"" Howlett",NaN
3,1868,Iron Man,https://marvel.fandom.com/wiki/Iron_Man_(Antho...,Public Identity,Good,Blue,Black,Male,NaN,Living,2961.0,Mar-63,1963.0,"Anthony \""Tony\"" Stark",NaN
4,2460,Thor,https://marvel.fandom.com/wiki/Thor_(Thor_Odin...,No Dual Identity,Good,Blue,Blond,Male,NaN,Living,2258.0,Nov-50,1950.0,Thor Odinson,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach,https://marvel.fandom.com/wiki/Ru%27ach_(Earth...,No Dual Identity,Evil,Green,No,Male,NaN,Living,NaN,NaN,NaN,Earth-616,NaN
16372,665474,Thane,https://marvel.fandom.com/wiki/Thane_(Thanos%2...,No Dual Identity,Good,Blue,Bald,Male,NaN,Living,NaN,NaN,NaN,Thanos' son,Earth-616)
16373,695217,Tinkerer,https://marvel.fandom.com/wiki/Tinkerer_(Skrul...,Secret Identity,Evil,Black,Bald,Male,NaN,Living,NaN,NaN,NaN,Skrull,Earth-616)
16374,708811,TK421,https://marvel.fandom.com/wiki/TK421_(Spiderli...,Secret Identity,Neutral,NaN,NaN,Male,NaN,Living,NaN,NaN,NaN,Spiderling,Earth-616)


In [149]:
df['ID']= df['ID'].str.split('I').str[0]


In [150]:
df["ID"] = df["ID"].replace(["No Dual "], "None")

In [151]:
df.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,Secret_Identity,Origin
0,1678,Spider-Man,https://marvel.fandom.com/wiki/Spider-Man_(Pet...,Secret,Good,Hazel,Brown,Male,NaN,Living,4043.0,Aug-62,1962.0,Peter Parker,NaN
1,7139,Captain America,https://marvel.fandom.com/wiki/Captain_America...,Public,Good,Blue,White,Male,NaN,Living,3360.0,Mar-41,1941.0,Steven Rogers,NaN
2,64786,Wolverine,https://marvel.fandom.com/wiki/Wolverine_(Jame...,Public,Neutral,Blue,Black,Male,NaN,Living,3061.0,Oct-74,1974.0,"James \""Logan\"" Howlett",NaN
3,1868,Iron Man,https://marvel.fandom.com/wiki/Iron_Man_(Antho...,Public,Good,Blue,Black,Male,NaN,Living,2961.0,Mar-63,1963.0,"Anthony \""Tony\"" Stark",NaN
4,2460,Thor,https://marvel.fandom.com/wiki/Thor_(Thor_Odin...,None,Good,Blue,Blond,Male,NaN,Living,2258.0,Nov-50,1950.0,Thor Odinson,NaN


In [152]:
def see_possible_categories(col):
    return df.groupby(col).sum()
     
see_possible_categories("ID")  

,page_id,APPEARANCES,Year
ID,,,
Known to Authorities,785553,1116.0,29893.0
None,720142592,41215.0,3165853.0
Public,1613083818,100208.0,8657706.0
Secret,1531859636,102871.0,11876655.0


## Some data is in the wrong format. For example, Year and APPEARANCES are floats but they really should be ints (as you cannot have a non-int number of appearances).

In [153]:
df["Year"]= df["Year"].astype('Int64')

In [154]:
df["APPEARANCES"]= df["APPEARANCES"].astype('Int64')

In [155]:
df.head(50)

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,Secret_Identity,Origin
0,1678,Spider-Man,https://marvel.fandom.com/wiki/Spider-Man_(Pet...,Secret,Good,Hazel,Brown,Male,NaN,Living,4043,Aug-62,1962,Peter Parker,NaN
1,7139,Captain America,https://marvel.fandom.com/wiki/Captain_America...,Public,Good,Blue,White,Male,NaN,Living,3360,Mar-41,1941,Steven Rogers,NaN
2,64786,Wolverine,https://marvel.fandom.com/wiki/Wolverine_(Jame...,Public,Neutral,Blue,Black,Male,NaN,Living,3061,Oct-74,1974,"James \""Logan\"" Howlett",NaN
3,1868,Iron Man,https://marvel.fandom.com/wiki/Iron_Man_(Antho...,Public,Good,Blue,Black,Male,NaN,Living,2961,Mar-63,1963,"Anthony \""Tony\"" Stark",NaN
4,2460,Thor,https://marvel.fandom.com/wiki/Thor_(Thor_Odin...,None,Good,Blue,Blond,Male,NaN,Living,2258,Nov-50,1950,Thor Odinson,NaN
5,2458,Benjamin Grimm,https://marvel.fandom.com/wiki/Benjamin_Grimm_...,Public,Good,Blue,No,Male,NaN,Living,2255,Nov-61,1961,Earth-616,NaN
6,2166,Reed Richards,https://marvel.fandom.com/wiki/Reed_Richards_(...,Public,Good,Brown,Brown,Male,NaN,Living,2072,Nov-61,1961,Earth-616,NaN
7,1833,Hulk,https://marvel.fandom.com/wiki/Hulk_(Robert_Br...,Public,Good,Brown,Brown,Male,NaN,Living,2017,May-62,1962,Robert Bruce Banner,NaN
8,29481,Scott Summers,https://marvel.fandom.com/wiki/Scott_Summers_(...,Public,Neutral,Brown,Brown,Male,NaN,Living,1955,Sep-63,1963,Earth-616,NaN
9,1837,Jonathan Storm,https://marvel.fandom.com/wiki/Jonathan_Storm_...,Public,Good,Blue,Blond,Male,NaN,Living,1934,Nov-61,1961,Earth-616,NaN


In [157]:
# if df["Year"] == "<NA>":
#     df["Year"] == df["APPEARANCES"]